ИУ5-62Б Лапотко Ирина. 
Задание: Задание. Для заданного набора данных построить модель регрессии . Для построения моделей используйте методы: Метод опорных векторов и Случайный лес. Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, mean_absolute_error, mean_squared_error, r2_score, accuracy_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline 

In [3]:
data = pd.read_csv('hotel_bookings.csv' , sep=",")
# Первые 5 строк датасета
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
data.shape

(119390, 32)

In [5]:
# проверим есть ли пропущенные значения
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [6]:
# Выберем числовые колонки с пропущенными значениями
# Цикл по колонкам датасета
total_count = data.shape[0]
num_cols = []
for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='float64' or dt=='int64'):
        num_cols.append(col)
        temp_perc = round((temp_null_count / total_count) * 100.0, 2)
        print('Колонка {}. Тип данных {}. Количество пустых значений {}, {}%.'.format(col, dt, temp_null_count, temp_perc))


Колонка children. Тип данных float64. Количество пустых значений 4, 0.0%.
Колонка agent. Тип данных float64. Количество пустых значений 16340, 13.69%.
Колонка company. Тип данных float64. Количество пустых значений 112593, 94.31%.


In [7]:
strategies=['mean', 'median', 'most_frequent']
def test_num_impute_col(dataset, column, strategy_param):
    temp_data = dataset[[column]]
    
    indicator = MissingIndicator()
    mask_missing_values_only = indicator.fit_transform(temp_data)
    
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(temp_data)
    
 
    filled_data = pd.DataFrame(data_num_imp, columns=[column])
    
    return column, strategy_param, mask_missing_values_only.sum(), filled_data[column][mask_missing_values_only[:, 0]].iloc[0], filled_data[column][mask_missing_values_only[:, 0]].iloc[-1]


In [8]:
imp_mean = SimpleImputer(strategy=strategies[0])
data['children'] = imp_mean.fit_transform(data[['children']])

In [9]:
imp_mean = SimpleImputer(strategy=strategies[0])
data['agent'] = imp_mean.fit_transform(data[['agent']])

In [10]:
imp_mean = SimpleImputer(strategy=strategies[0])
data['company'] = imp_mean.fit_transform(data[['company']])

In [11]:
data.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [12]:
# Кодирование категориальных признаков
label_encoder = LabelEncoder()
data['hotel'] = label_encoder.fit_transform(data['hotel'])
data['arrival_date_month'] = label_encoder.fit_transform(data['arrival_date_month'])
data['meal'] = label_encoder.fit_transform(data['meal'])
data['country'] = label_encoder.fit_transform(data['country'])
data['market_segment'] = label_encoder.fit_transform(data['market_segment'])
data['distribution_channel'] = label_encoder.fit_transform(data['distribution_channel'])
data['reserved_room_type'] = label_encoder.fit_transform(data['reserved_room_type'])
data['assigned_room_type'] = label_encoder.fit_transform(data['assigned_room_type'])
data['deposit_type'] = label_encoder.fit_transform(data['deposit_type'])
data['reservation_status'] = label_encoder.fit_transform(data['reservation_status'])
data['reservation_status_date'] = label_encoder.fit_transform(data['reservation_status_date'])
data['customer_type'] = label_encoder.fit_transform(data['customer_type'])

In [13]:
X = data.drop('lead_time', axis=1)
y = data['lead_time']


In [14]:
# разделение на тестовую и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# метод опорных векторов
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Обучение модели
svr_model = SVR()
svr_model.fit(X_train, y_train)

# Предсказание
y_pred_svr = svr_model.predict(X_test)

# Оценка качества модели
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print(f'SVR MSE: {mse_svr}')
print(f'SVR R2: {r2_svr}')


In [15]:

# Обучение модели
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Предсказание
y_pred_rf = rf_model.predict(X_test)

# Оценка качества модели
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f'Random Forest MSE: {mse_rf}')
print(f'Random Forest R2: {r2_rf}')


На основе значений MSE и R2 можно сделать вывод о качестве моделей. Например, если модель случайного леса показывает более низкий MSE и более высокий R2 по сравнению с моделью опорных векторов, можно заключить, что случайный лес лучше справляется с задачей предсказания lead_time.
Таким образом, метод случайного леса предпочтительней для данной задачи.